In [ ]:
using GLMakie, LinearAlgebra, Printf

# Define transition matrix 
   # Nominal  Micro cracks Partial Delaminated Failed
P = [
        0.95       0.00           0.00         0.00;        # Nominal
        0.04       0.88           0.00         0.00;        # Micro cracks
        0.01       0.10           0.8          0.00;        # Partial Delaminated
        0.00       0.02           0.2          1.00]        # Failed

# Initial state (all probability in 'Nominal')
x_today = [1.0, 0.0, 0.0, 0.0]
n_weeks = 200
states = ["Nominal", "Micro-cracks", "Partial Delaminated", "Failed"]

# Evolve Markov chain over time
history = zeros(4, n_weeks + 1)
history[:, 1] = x_today
for k in 1:n_weeks
    x_today = P * x_today
    history[:, k+1] = x_today
end

# Time axis
weeks = 0:n_weeks
colors = [:green, :orange, :purple, :red]

# Setup Makie plot
fig = Figure(size = (1000, 500))
ax = Axis(fig[1, 1], xlabel = "Weeks in Orbit", ylabel = "State Probability", title = "Satellite Degradation Probability Over Time with Marcov Chain")

lines = [lines!(ax, weeks, history[i, :], color = colors[i], label = states[i], linewidth=2) for i in 1:4]
dots = [scatter!(ax, [0.0], [history[i, 1]], color = colors[i], markersize=10) for i in 1:4]
axislegend(ax; position = :rc, framevisible = false)  
vline = vlines!(ax, [0.0], color = :gray, linestyle = :dash)

# Labels
current_week = Observable(0)
info_label = Label(fig[0, 1], "Week: 0   Most Likely: Nominal", fontsize=18, color=:black, halign=:center)

# Update function
on(current_week) do w
    w_idx = clamp(w + 1, 1, length(weeks))
    for i in 1:4
        dots[i][1][] = [Point2f(weeks[w_idx], history[i, w_idx])]
    end
    vline[1][] = [weeks[w_idx]]
    
    # Determine most probable state
    probs = history[:, w_idx]
    max_idx = argmax(probs)
    @printf("Week %3d → %.4f %.4f %.4f %.4f\n", w, probs...)
    formatted_probs = join([@sprintf("%s: %.3f", states[i], probs[i]) for i in 1:4], "   ")

    info_label.text[] = "Week: $w     $formatted_probs"
end

# Record animation
record(fig, "satellite_markov_chain.mp4", 0:n_weeks; framerate=15) do w
    current_week[] = w
end
